# How to run a semantic skills from file
Now that you're familiar with Kernel basics, let's see how the kernel allows you to run Semantic Skills and Semantic Functions stored on disk. 

A Semantic Skill is a collection of Semantic Functions, where each function is defined with natural language that can be provided with a text file. 

Refer to our [glossary](https://github.com/microsoft/semantic-kernel/blob/main/docs/GLOSSARY.md) for an in-depth guide to the terms.

The repository includes some examples under the [samples](https://github.com/microsoft/semantic-kernel/tree/main/samples) folder.

For instance, [this](../../skills/FunSkill/Joke/skprompt.txt) is the **Joke function** part of the **FunSkill skill**:

```
WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE TOPIC BELOW.
JOKE MUST BE:
- G RATED
- WORKPLACE/FAMILY SAFE
NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY.
BE CREATIVE AND FUNNY. I WANT TO LAUGH.
+++++
{{$input}}
+++++
```

Note the special **`{{$input}}`** token, which is a variable that is automatically passed when invoking the function, commonly referred to as a "function parameter". 

We'll explore later how functions can accept multiple variables, as well as invoke other functions.


In the same folder you'll notice a second [config.json](../../skills/FunSkill/Joke/config.json) file. The file is optional, and is used to set some parameters for large language models like Temperature, TopP, Stop Sequences, etc.

```
{
  "schema": 1,
  "type": "completion",
  "description": "Generate a funny joke",
  "completion": {
    "max_tokens": 500,
    "temperature": 0.5,
    "top_p": 0.5
  }
}
```

Given a semantic function defined by these files, this is how to load and use a file based semantic function.

Load and configure the kernel, as usual, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):

In [ ]:
# !python -m pip install semantic-kernel==0.4.3.dev0

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    azure_chat_service = AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key)   # set the deployment name to the value of your chat model
    kernel.add_chat_service("chat_completion", azure_chat_service)
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    oai_chat_service = OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id)
    kernel.add_chat_service("chat-gpt", oai_chat_service)

Import the skill and all its functions:

In [2]:
# note: using skills from the samples folder
skills_directory = "../../samples/skills"

funFunctions = kernel.import_semantic_skill_from_directory(skills_directory, "FunSkill")

jokeFunction = funFunctions["Joke"]

How to use the skill functions, e.g. generate a joke about "*time travel to dinosaur age*":

In [11]:
context_variables = sk.ContextVariables(variables={
    "input": "knock, knock!",
    "style": "chickens"
})
result = jokeFunction(variables=context_variables)

print(result)

# You can also invoke functions asynchronously
# result = await jokeFunction.invoke_async("time travel to dinosaur age")
# print(result)

Who's there?
Egg.
Egg who?
Eggcuse me, but do you have any chicken jokes? I'm feeling a little cooped up here!


In [14]:
print(context_variables.variables)

{'input': "Who's there?\nEgg.\nEgg who?\nEggcuse me, but do you have any chicken jokes? I'm feeling a little cooped up here!", 'style': 'chickens'}


In [9]:
result = await jokeFunction.invoke_async("knock, knock!")

Variable `$style` not found


In [10]:
print(result)

Who's there, matey?
+++++
Lettuce.
+++++
Lettuce who?
+++++
Lettuce in, it's cold out here and I'm freezing me booty off!


In [12]:
type(jokeFunction)

semantic_kernel.orchestration.sk_function.SKFunction